<a href="https://colab.research.google.com/github/hml-yt/colabs/blob/main/HML-InstructPix2Pix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# InstructPix2Pix: Learning to Follow Image Editing Instructions

Easy to launch Colab Notebook for running Instruct Pix2Pix, with added extensions (ControlNet, Deforum). Made for the viewers of the "[Hacking Modern Life](https://youtube.com/@hml)" YouTube Channel by David Tabachnikov. Original InstructPix2Pix by Timothy Brooks model is available [here](https://github.com/timothybrooks/instruct-pix2pix), and the paper is [here](https://arxiv.org/abs/2211.09800).

<img src='https://instruct-pix2pix.timothybrooks.com/teaser.jpg'/>

In [ ]:
#@title Install Dependencies
from IPython.utils import capture
from IPython.display import clear_output
import ipywidgets as widgets

def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)

with capture.capture_output() as cap:
  !pip install -qqq git+https://github.com/huggingface/diffusers.git gradio transformers accelerate safetensors
  !wget https://raw.githubusercontent.com/hml-yt/colabs/main/InstructPix2Pix/sample.jpg

In [ ]:
#@title Setup UI

from __future__ import annotations

import math
import random

import gradio as gr
import torch
from PIL import Image, ImageOps
from diffusers import StableDiffusionInstructPix2PixPipeline


help_text = """
If you're not getting what you want, there may be a few reasons:
1. Is the image not changing enough? Your Image CFG weight may be too high. This value dictates how similar the output should be to the input. It's possible your edit requires larger changes from the original image, and your Image CFG weight isn't allowing that. Alternatively, your Text CFG weight may be too low. This value dictates how much to listen to the text instruction. The default Image CFG of 1.5 and Text CFG of 7.5 are a good starting point, but aren't necessarily optimal for each edit. Try:
    * Decreasing the Image CFG weight, or
    * Increasing the Text CFG weight, or
2. Conversely, is the image changing too much, such that the details in the original image aren't preserved? Try:
    * Increasing the Image CFG weight, or
    * Decreasing the Text CFG weight
3. Try generating results with different random seeds by setting "Randomize Seed" and running generation multiple times. You can also try setting "Randomize CFG" to sample new Text CFG and Image CFG values each time.
4. Rephrasing the instruction sometimes improves results (e.g., "turn him into a dog" vs. "make him a dog" vs. "as a dog").
5. Increasing the number of steps sometimes improves results.
6. Do faces look weird? The Stable Diffusion autoencoder has a hard time with faces that are small in the image. Try:
    * Cropping the image so the face takes up a larger portion of the frame.
"""


example_instructions = [
    "Make it a picasso painting",
    "as if it were by modigliani",
    "convert to a bronze statue",
    "Turn it into an anime.",
    "have it look like a graphic novel",
    "make him fat",
    "what would he look like bald?",
    "Have him smile",
    "Put him in a cocktail party.",
    "put him at the beach",
    "Convert to black and white",
    "What if it were snowing?",
    "Give him a leather jacket",
    "Turn him into a cyborg!",
    "make him wear a beanie",
]

pipe = StableDiffusionInstructPix2PixPipeline.from_pretrained(
    "timbrooks/instruct-pix2pix",
    torch_dtype=torch.float16,
    revision="fp16",
    safety_checker=None,
).to("cuda")
example_image = Image.open("/content/IMG_4703.jpg").convert("RGB")


def load_example(
    steps: int,
    randomize_seed: bool,
    seed: int,
    randomize_cfg: bool,
    text_cfg_scale: float,
    image_cfg_scale: float,
):
    example_instruction = random.choice(example_instructions)
    return [example_image, example_instruction] + generate(
        example_image,
        example_instruction,
        steps,
        randomize_seed,
        seed,
        randomize_cfg,
        text_cfg_scale,
        image_cfg_scale,
    )


def generate(
    input_image: Image.Image,
    instruction: str,
    steps: int,
    randomize_seed: bool,
    seed: int,
    randomize_cfg: bool,
    text_cfg_scale: float,
    image_cfg_scale: float,
):
    seed = random.randint(0, 100000) if randomize_seed else seed
    text_cfg_scale = (
        round(random.uniform(6.0, 9.0), ndigits=2) if randomize_cfg else text_cfg_scale
    )
    image_cfg_scale = (
        round(random.uniform(1.2, 1.8), ndigits=2) if randomize_cfg else image_cfg_scale
    )

    width, height = input_image.size
    factor = 512 / max(width, height)
    factor = math.ceil(min(width, height) * factor / 64) * 64 / min(width, height)
    width = int((width * factor) // 64) * 64
    height = int((height * factor) // 64) * 64
    input_image = ImageOps.fit(input_image, (width, height))

    if instruction == "":
        return [input_image, seed]

    generator = torch.manual_seed(seed)
    edited_image = pipe(
        instruction,
        image=input_image,
        guidance_scale=text_cfg_scale,
        image_guidance_scale=image_cfg_scale,
        num_inference_steps=steps,
        generator=generator,
    ).images[0]
    return [seed, text_cfg_scale, image_cfg_scale, edited_image]


def reset():
    return [0, "Randomize Seed", 1371, "Fix CFG", 7.5, 1.5, None]


with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column(scale=1, min_width=100):
            generate_button = gr.Button("Generate")
        with gr.Column(scale=1, min_width=100):
            load_button = gr.Button("Load Example")
        with gr.Column(scale=1, min_width=100):
            reset_button = gr.Button("Reset")
        with gr.Column(scale=3):
            instruction = gr.Textbox(
                lines=1, label="Edit Instruction", interactive=True
            )

    with gr.Row():
        input_image = gr.Image(label="Input Image", type="pil", interactive=True)
        edited_image = gr.Image(label=f"Edited Image", type="pil", interactive=False)
        input_image.style(height=512, width=512)
        edited_image.style(height=512, width=512)

    with gr.Row():
        steps = gr.Number(value=10, precision=0, label="Steps", interactive=True)
        randomize_seed = gr.Radio(
            ["Fix Seed", "Randomize Seed"],
            value="Randomize Seed",
            type="index",
            show_label=False,
            interactive=True,
        )
        seed = gr.Number(value=1371, precision=0, label="Seed", interactive=True)
        randomize_cfg = gr.Radio(
            ["Fix CFG", "Randomize CFG"],
            value="Fix CFG",
            type="index",
            show_label=False,
            interactive=True,
        )
        text_cfg_scale = gr.Number(value=7.5, label=f"Text CFG", interactive=True)
        image_cfg_scale = gr.Number(value=1.5, label=f"Image CFG", interactive=True)

    gr.Markdown(help_text)

    load_button.click(
        fn=load_example,
        inputs=[
            steps,
            randomize_seed,
            seed,
            randomize_cfg,
            text_cfg_scale,
            image_cfg_scale,
        ],
        outputs=[
            input_image,
            instruction,
            seed,
            text_cfg_scale,
            image_cfg_scale,
            edited_image,
        ],
    )
    instruction.submit(
        fn=generate,
        inputs=[
            input_image,
            instruction,
            steps,
            randomize_seed,
            seed,
            randomize_cfg,
            text_cfg_scale,
            image_cfg_scale,
        ],
        outputs=[seed, text_cfg_scale, image_cfg_scale, edited_image],
    )
    generate_button.click(
        fn=generate,
        inputs=[
            input_image,
            instruction,
            steps,
            randomize_seed,
            seed,
            randomize_cfg,
            text_cfg_scale,
            image_cfg_scale,
        ],
        outputs=[seed, text_cfg_scale, image_cfg_scale, edited_image],
    )
    reset_button.click(
        fn=reset,
        inputs=[],
        outputs=[
            steps,
            randomize_seed,
            seed,
            randomize_cfg,
            text_cfg_scale,
            image_cfg_scale,
            edited_image,
        ],
    )

In [ ]:
#@title Start/Stop Instruct Pix2Pix
demo.launch()